In [114]:
import gist ### https://github.com/tuttieee/lear-gist-python

from PIL import Image
import imageio
from IPython.display import display, clear_output

import sys
sys.path.append("../Library/")
import image_download as imd
import image_manipulation as ima

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
load_dotenv("../.env")
import csv

import numpy as np
import db_connection as dbcon

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score


#%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Set Parameters

In [115]:
# parameters
zoom = 15
lat = 30.4131736
lon = -87.600026
pixels = 640
samples_per_location = 1
precision = 4
num_images = 4
GMAPS_API_KEY = os.getenv('GMAPS_API_KEY')
IMG_FOLDER = os.getenv('MFP_IMG_FOLDER')

# connect to db
db = dbcon.connect("../credentials/mlab_db.txt","mfp")
images_lib_col = db["images_lib"]

# image dimensions
meters_per_px = imd.zoom_in_meters_per_pixel(zoom, lat)
image_size = meters_per_px*pixels ## in meters

print("Meters per pixel:", round(meters_per_px,2))
print("Image size:", round(image_size,2), "in meters")

DB Credentials from file
DB connected successfully!!!
Meters per pixel: 4.12
Image size: 2636.76 in meters


### Get filenames of labelled images from db

In [116]:
labelled_images = dbcon.query_filenames_of_labelled_images(images_lib_col, "label_binary_pw")
labelled_images["label_multi"] = dbcon.query_filenames_of_labelled_images(images_lib_col, "label_multi_pw")["label"]

# Convert label to integer
labelled_images["label"] = labelled_images["label"].astype(int)
labelled_images["label_multi"] = labelled_images["label_multi"].astype(int)

# Remove files with label_multi = 10. These are files where Google didn't have an image
# at the provided coordinate
labelled_images = labelled_images[labelled_images.label_multi <= 4]


labelled_images.head()

,filename,label,label_multi
0,US1_45.21160965529817_-93.54554509042913_13_64...,1,3
1,US1_45.21160965529817_-93.54554509042913_14_64...,1,4
2,US1_45.21160965529817_-93.54554509042913_15_64...,1,3
3,US1_45.21160965529817_-93.54554509042913_16_64...,1,3
4,US1_52.20482183860124_-92.48891258998856_13_64...,0,0


### Load labelled images from Google Drive

In [117]:
images_info = ima.load_images_from_gdrive(labelled_images.filename, IMG_FOLDER)

In [118]:
# Construct df with np.array of images in one column
labelled_images['img_array'] = images_info['array']
# Convert label to integer
labelled_images["label"] = labelled_images["label"].astype(int)

print("There are", labelled_images.shape[0], "labelled images in the dataset!")
labelled_images.head()

There are 57 labelled images in the dataset!


,filename,label,label_multi,img_array
0,US1_45.21160965529817_-93.54554509042913_13_64...,1,3,"[[[42, 57, 41], [20, 40, 24], [12, 36, 20], [1..."
1,US1_45.21160965529817_-93.54554509042913_14_64...,1,4,"[[[42, 49, 34], [42, 49, 34], [42, 49, 34], [4..."
2,US1_45.21160965529817_-93.54554509042913_15_64...,1,3,"[[[68, 64, 48], [89, 88, 68], [57, 57, 43], [3..."
3,US1_45.21160965529817_-93.54554509042913_16_64...,1,3,"[[[70, 68, 59], [70, 68, 59], [93, 89, 82], [9..."
4,US1_52.20482183860124_-92.48891258998856_13_64...,0,0,"[[[10, 36, 36], [10, 36, 36], [10, 36, 36], [1..."


In [119]:
# Generate feature vector with gist
feature_vector = []
for image in labelled_images["img_array"].tolist():
    feature_vector.append(gist.extract(image))
    
feature_matrix = np.array(feature_vector)

### Classification on binary label

In [135]:
X_train, X_test, y_train, y_test = train_test_split(feature_matrix, labelled_images.label, test_size = 0.25)

In [136]:
clf = LinearSVC()# for multi class labels use multi_class="crammer_singer"

In [137]:
clf.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [138]:
y_pred = clf.predict(X_test)

In [139]:
accuracy_score(y_test, y_pred)

0.8666666666666667